## Clause Risk Categorization

Putting each of the TCLP clauses into risk categories using an LLM 

In [113]:
import pandas as pd
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
import utils

In [114]:
risk_taxonomy = pd.read_excel('../data/risk_taxonomy.xlsx')

In [115]:
clause_folder = "../data/cleaned_content"
clause_html = '../data/clause_boxes'
model_path = "../models/CC_BERT/CC_model_detect"

In [116]:
tokenizer, d_model, c_model, names, docs, final_df = utils.getting_started(model_path, clause_folder, clause_html)

/Users/georgia/Documents/coding/climate_risk_id/tclp/utils.py:557: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(content, "html.parser")


In [117]:
# make a df of names and docs
df = pd.DataFrame({'name': names, 'clause': docs})

In [118]:
risk_taxonomy

,Label,Description
0,Physical-flooding,Clause that helps reduce exposure to flooding ...
1,Physical-wildfire,Clause that helps mitigate exposure to wildfir...
2,Physical-heat,Clause that helps reduce exposure to overheati...
3,Physical-subsidence,Clause that helps reduce exposure to ground in...
4,Physical-sea-level,Clause that helps reduce exposure to coastal e...
5,Physical-water-scarcity,Clause that helps reduce exposure to water str...
6,Physical-extreme-weather,"Clause that helps reduce exposure to storm, wi..."
7,Physical-infrastructure,Clause that helps reduce exposure to infrastru...
8,Physical-general,Clause that helps manage general exposure to p...
9,Transition-mees,Clause that helps reduce exposure to MEES-rela...


In [119]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"), 
    base_url = "https://openrouter.ai/api/v1"
)

In [120]:
messages = "You are a helpful assistant whose job it is to identify the risk type given a provided clause. These clauses WILL NOT contain the risk themselves. Rather, they are designed to help legal users to mitigate risk. So you are meant to identify the risk categorizations that the given clause might help protect against. Feel free to pick more than one risk that you think the clause could be relevant for."

In [121]:
clause_1 = df.iloc[0]['name'] + df.iloc[0]['clause'] 

In [122]:
result = utils.classify_clause(clause_1, risk_taxonomy, messages, client)

In [123]:
print(result)

{
  "labels": ["Transition-retrofit", "Transition-standards", "Transition-disclosure"],
  "justification": "The clause provides a detailed guide and checklist for accessing Sustainability-Linked Loans (SLLs), which helps companies align their financial strategies with net zero transition goals. It includes setting sustainability performance targets (SPTs), reporting on progress, and ensuring these targets are ambitious and meaningful. This helps mitigate risks related to future retrofit obligations (Transition-retrofit) by encouraging companies to take proactive steps to improve their sustainability performance. It also addresses the need to align with voluntary standards and frameworks (Transition-standards), such as the LMA Sustainability Linked Loan Principles, which can help avoid reputational risks. Additionally, the clause emphasizes the importance of disclosure and reporting, which can help reduce exposure to legal claims related to failure to disclose known or foreseeable clima

____

## Creating database and applying this to all clauses

In [124]:
# Get the list of all possible risk labels (from your taxonomy)
risk_labels = list(risk_taxonomy['Label'].str.strip())

In [125]:
results_df = pd.DataFrame(columns=['name'] + risk_labels + ['justification'])

In [147]:
def perform_risk_categorization(df, results_df): 
    for i, row in df.iterrows():
        clause_text = row['name'] + row['clause']
        result = utils.classify_clause(clause_text, risk_taxonomy, messages, client)
        
        print(f"Processing clause {i+1}/{len(df)}: {row['name']}")
        
        # Format the result
        formatted_row = utils.format_classification_result(row['name'], result, risk_labels)
        
        # Append to the DataFrame
        results_df = pd.concat([results_df, pd.DataFrame([formatted_row])], ignore_index=True)
    
    return results_df

In [154]:
def post_process_results(results_df):
    results_df[risk_labels] = results_df[risk_labels].astype(int)
    results_df['total_risks'] = results_df[risk_labels].sum(axis=1)
    results_df['has_risk_flag'] = results_df[risk_labels].any(axis=1).map({True: 'TRUE', False: 'FALSE'})
    results_df['combined_labels'] = results_df[risk_labels] \
    .eq(1) \
    .apply(lambda mask: ', '.join(mask.index[mask]), axis=1)
    return results_df

In [146]:
results_df.has_risk_flag.value_counts()

has_risk_flag
TRUE     103
FALSE     19
Name: count, dtype: int64

____

# Doing this again for the ones that were skipped the first time

In [166]:
false_df = results_df[results_df['has_risk_flag'] == 'FALSE']

In [167]:
zeros_list = false_df['name'].tolist()

#subset the original df to only those clauses that have no labels
df_0s = df[df['name'].isin(zeros_list)]

In [168]:
new_results = pd.DataFrame(columns=['name'] + risk_labels + ['justification'])

In [169]:
new_results = perform_risk_categorization(df_0s, new_results)

Processing clause 15/2: Climate-Related Disclosure in Loans
Processing clause 104/2: Stakeholder Company Climate Questionnaire


In [170]:
new_results = post_process_results(new_results)

In [171]:
orig = results_df.set_index('name')
updates = new_results.set_index('name')

# 2. Overwrite only those rows in `orig` where `updates` has data
orig.update(updates)

# 3. Bring `name` back as a column
results_df = orig.reset_index()

In [172]:
results_df.has_risk_flag.value_counts()

has_risk_flag
TRUE    122
Name: count, dtype: int64

In [173]:
#save the results
results_df.to_csv('../data/risk_categorization_results.csv', index=False)